<a href="https://colab.research.google.com/github/RO-HIT17/CodeReview-Kuriyam/blob/main/CodeLLama_Finetuning_CyberNative.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
  !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
  !pip install sentencepiece protobuf huggingface_hub hf_transfer
  !pip install transformers==4.51.3
  !pip install --no-deps unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True


fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
]

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/codellama-7b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,

)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

In [ ]:
!pip install -U datasets
from datasets import load_dataset
dataset = load_dataset("CyberNative/Code_Vulnerability_Security_DPO", split="train")
print(dataset.column_names)

In [ ]:
dataset = to_sharegpt(
    dataset,
    merged_prompt=(
        "{question}\n\n"
        "The following response was rejected due to a security vulnerability:\n"
        "{rejected}\n\n"
        "Reason: {vulnerability}\n\n"
        "Please provide a better and secure alternative."
    ),
    output_column_name="chosen",
    conversation_extension=3
)


dataset = standardize_sharegpt(dataset)

chat_template = """USER:
{INPUT}

ASSISTANT:
{OUTPUT}"""

from unsloth import apply_chat_template

dataset = apply_chat_template(
    dataset,
    tokenizer=tokenizer,
    chat_template=chat_template
)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,

        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

In [ ]:
trainer_stats = trainer.train()

In [ ]:
from unsloth import FastLanguageModel
from transformers import TextStreamer

FastLanguageModel.for_inference(model)

messages = [
    {
        "role": "user",
        "content": (
            "Write a function to hash a password.\n\n"
            "The following response was rejected due to a security vulnerability:\n"
            "```python\n"
            "def hash_password(password):\n"
            "    return hashlib.md5(password.encode()).hexdigest()\n"
            "```\n\n"
            "Reason: Use of MD5 which is cryptographically broken and not recommended for password hashing.\n\n"
            "Please provide a better and secure alternative."
        )
    }
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

text_streamer = TextStreamer(tokenizer, skip_prompt=True)

_ = model.generate(
    input_ids,
    streamer=text_streamer,
    max_new_tokens=256,
    pad_token_id=tokenizer.eos_token_id,
)


In [ ]:
model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
if True: model.save_pretrained_gguf("model", tokenizer,)

In [ ]:
import subprocess

subprocess.Popen(["ollama", "serve"])
import time

time.sleep(3)

In [ ]:
print(tokenizer._ollama_modelfile)
!ollama create unsloth_model -f ./model/Modelfile

In [ ]:
from google.colab import files


!zip -r lora_model.zip lora_model
files.download("lora_model.zip")